In [4]:
import pandas as pd
import numpy as np
import data_prep
import TensorRecommend


games_info = pd.read_csv("C:\\Users\\dijan\\Documents\\GitHub\\Reccomender-system-for-board-games\\data\\raw\\games_detailed_info2025.csv")
games_info.rename(columns={'id': 'ID'}, inplace=True)
bgg_reviews = pd.read_csv("C:\\Users\\dijan\\Documents\\GitHub\\Reccomender-system-for-board-games\\data\\raw\\bgg-26m-reviews.csv")

data_preprocessor = data_prep.DataPreprocessor(games_info, bgg_reviews)
game_info_cut = data_preprocessor.discretize_column(games_info, 'usersrated', n_bins=10, percent=0.25, drop_original=True )
game_info_final = data_preprocessor.discretize_column(game_info_cut, 'yearpublished', n_bins=10, drop_original=True)



In [7]:
data_preprocessor = data_prep.DataPreprocessor(game_info_final, bgg_reviews)


In [5]:
game_info_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6945 entries, 0 to 6944
Data columns (total 52 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     6945 non-null   int64  
 1   type                           6945 non-null   object 
 2   ID                             6945 non-null   int64  
 3   thumbnail                      6945 non-null   object 
 4   image                          6945 non-null   object 
 5   alternate                      4732 non-null   object 
 6   description                    6945 non-null   object 
 7   minplayers                     6945 non-null   int64  
 8   maxplayers                     6945 non-null   int64  
 9   suggested_num_players          6945 non-null   object 
 10  suggested_playerage            6945 non-null   object 
 11  suggested_language_dependence  6945 non-null   object 
 12  playingtime                    6945 non-null   int64 

In [8]:

merged_df = data_preprocessor.merge_datasets(featires_to_add=['usersrated_bin', 'yearpublished_bin', 'suggested_num_players'])


In [ ]:

data_entries, num_users, num_items, num_features, user2idx, game2idx, feature2idx = data_preprocessor.data_to_tuple(merged_df, user_col='user', item_col='ID', feature_col=['usersrated_bin', 'yearpublished_bin', 'suggested_num_players'], rating_col='rating')

data_train, data_test = data_preprocessor.train_test_split(data_entries, test_size=0.3)

epochs = 10
k = 5
lambda_ = 0.1
tessst = TensorRecommend.TensorRecommend(k, lambda_=lambda_, eta=0.01, data_entries=data_train, num_users=num_users, 
                                             num_items=num_items, num_features=num_features)


for epoch in range(epochs):
    tessst.train_epoch(t=epoch+1, initial_lr=0.01, max_norm=5.0)

np.save("U.npy", tessst.U)
np.save("M.npy", tessst.M)
for i, C_mat in enumerate(tessst.C):
    np.save(f"C{i+1}.npy", C_mat)
np.save("S.npy", tessst.S)

In [16]:
sample = data_train[0]

tessst.predict((0,0,0,0,0))

np.float64(8.292249977448622)